In [1]:
import os
from datasets import load_dataset

c:\Users\juan1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
def coco_to_yolo(x, y, w, h, width, heigth):
    return [((2 * x + w) / (2 * width)), ((2 * y + h) / (2 * heigth)), w / width, h / heigth]

In [14]:
def preprocessing(partition: str, data :object):
    os.makedirs(f"datasets/images/{partition}", exist_ok=True)
    os.makedirs(f"datasets/labels/{partition}", exist_ok=True)

    # data["train"][0] 
    for i, sample in enumerate(data[partition]):
        image = sample['image']
        label = sample["objects"]['category']
        bboxes = sample["objects"]['bbox']
        width = sample['width']
        height = sample['height']

        # processing bboxes
        items = []
        for label, bbox in zip(label, bboxes):
            xc, yc, w, h = coco_to_yolo(bbox[0], bbox[1], bbox[2], bbox[1], width, height)
            items.append(f"{label} {xc} {yc} {w} {h}")
        
        with open(f"datasets/labels/{partition}/{i}.txt", "w") as f:
            for item in items:
                f.write(f"{item}\n")

        image.save(f"datasets/images/{partition}/{i}.jpg")

In [15]:
if (__name__ == "__main__"):
    data = load_dataset("https://universe.roboflow.com/ds/OpmEyv9HEN?key=aT00D1ddg4", "full")

    preprocessing("train", data)
    preprocessing("test", data)
    preprocessing("validation", data)